In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime as dt
import csv 
import psycopg2
import time

In [2]:
# Get HTML code we want to parse.  This HTML link is a search on Craigslist for the term "math tutor"

current_region = 'dallas'

response = get('https://dallas.craigslist.org/d/services/search/bbb?query=math%20tutor&sort=rel')

# Parse as BeautifulSoup object using html.parser
html_soup = BeautifulSoup(response.text, 'html.parser')

# Find all posts to scrape relevant information from later
posts = html_soup.find_all('li', class_='result-row')

In [3]:
# Figuring out how to get the link of each search result to scrape from
post_one = posts[0]
post_one.a

<a class="result-image gallery empty" href="https://dallas.craigslist.org/dal/lss/d/dallas-thealert-mathtutor/7425156915.html"></a>

In [737]:
# Create a list of soup_objects to scrape from by storing them into memory.  This way we only have to do this once and Craigslist doesn't get mad at us (ban us) for sending too many https requests

soup_objects_list = []
link_list = []
for post in posts:
    current_link = post.a.get('href')
    link_list.append(current_link)
    response_object = get(current_link)
    soup_object = BeautifulSoup(response_object.text, 'html.parser')
    soup_objects_list.append(soup_object)

In [759]:
# Initialize several lists to store relevant information for analysis
price_list = []
city_list = []
datetime_list = []
body_text_list = []
subregion_list = []
region_list = []


# Walk through each soup object in the list
for soup in soup_objects_list:
    
    # Extract region of Craigslist
    # This will allow for comparison of prices across different
    # states or metropolitan areas
    ### We do this FIRST because Craigslist shows postings from neighboring areas as 
    ### the bottom of the search page.  If we compare the region against 
    ### current_region, it will ensure only the results from the region we care about 
    ### get into our dataframe later
    region = soup.find_all('li',class_='crumb area')[0].find('a').get_text()
    region = region.replace(' ', '_')
    region = region.lower()
    if region == current_region:
        region_list.append(region)




        # Get text of postingbody of the post and remove unwanted text
        text = soup.find('section', id='postingbody').get_text()
        text = text.replace('\n', '')
        text = text.replace(';', ',') # We do this so that we can use ; as a 
                                      # delimiter when copying data from a CSV file 
                                      # into a SQL database later
        text = text.replace('QR Code Link to This Post', '') # We do this because this
                                                             # text from one post in 
                                                             # particular was giving me 
                                                             # trouble and I couldn't  
                                                             # figure out a better way to 
                                                             # handle it
        body_text_list.append(text)



        # Use regular expressions to find all instances of prices in the text
        #old_prices = re.findall('(?:[\$]{1}[,\d]+.?\d*)', text)
        old_prices = re.findall('(?:[\$]{1}[,\d]+\d*)', text)
        # Intialize empty list to store the new prices after processing old prices
        new_prices = []

        # Walk through each price in the post
        for price in old_prices:
            # Clean unwanted characters
            price = price.replace('$', '')
            price = price.replace('/', '')
            price = price.replace('!', '')
            price = price.replace('h', '')
            price = price.replace('.', '')
            price = price.replace(')', '')
            price = price.replace(',', '')
            price = price.replace('>', '')
            price = price.rstrip()   
            # Some tutors give prices as a range ie '$30-40'.  In order to
            # work with this data, I split based on the hyphen, then can 
            # use each price individually
            split_prices = price.split('-')

            # Walk through each price in the posting, including one's that have been               split
            for p in split_prices:
                # Only proceed if p is a non-empty string
                if len(p)!=0:
                    # Convert string price to int
                    try:
                        # Ignore prices over 150, which are far too high to be reasonable
                        new_int = int(p)
                        if new_int <= 150:
                            new_prices.append(new_int)
                    except:
                        # Show which prices aren't able to convert to an int and 
                        # the post they came from
                        print(F'Error converting this price: {p}')
                        print(old_prices)
                        print()
                        print('Here is the text of the post:')
                        print()
                        print(text)
                        print('-'*50)
                        print()
                        new_prices.append(np.nan) # set prices that can't be covered to NaN

        # For posts that had no prices listed, we append new_prices with "None"
        if len(new_prices)==0:
            price_list.append('None')
        # For posts that had a single price, we use it
        elif len(new_prices)==1:
            price_list.append(new_prices[0])
        # For posts that contained two prices, we average them
        elif len(new_prices)==2:
            avg_price_2 = np.average(new_prices)
            price_list.append(avg_price_2)
        # If a post has more than 3 prices, we append them, but this means we have to           inspect them and deal with them manually later
        else:
            price_list.append(new_prices)


        # Get city information for each posting
        try:
            city = soup.find(class_='postingtitletext').small.get_text()
            re_pattern = re.compile('online')
            online_flag = re.search(re_pattern, city.lower())
            if online_flag:
                city_list.append('Online')
            else:
                # Strip out leading and trailing white spaces, replace parentheses, and                 capitalize each word in the str
                city = city.strip()
                city = city.replace('(', '').replace(')', '')        
                city = city.title()
                city_list.append(city)
        except:
            # If a post has no city information, use None
            city_list.append('None')

        # Extract subregion of Craigslist that post was made
        # This will allow for comparison of prices across different cities
        # within the same metropolitan sub_region
        try:
            subregion = soup.find_all('li', class_='crumb subarea')[0].find('a').get_text()
            subregion = subregion.title()
            subregion_list.append(subregion)
        except:
            subregion_list.append('None')


        # Extract time the posting was made
        try:
            dt_object = soup.find('time')['datetime']
            datetime_list.append(dt_object)
        except:
            datetime_list.append('None')
    else:
        pass

In [760]:
# Get date of html request to label our output
date_of_html_request = str(dt.date.today())

# Create dataframe of our results

df = pd.DataFrame(data=zip(datetime_list, link_list, price_list, city_list, subregion_list, region_list, body_text_list),
                  columns=['date_posted', 'link', 'price', 'city', 'subregion', 'region', 'post_text'])

# Keep track of which prices are still lists, we will need to investigate these later
df['prices_need_cleaning'] = df['price'].apply(lambda x: isinstance(x, list) and len(x) >= 3)

# Include date posts were scraped on to track tutoring prices over time
df['posts_scraped_on'] = date_of_html_request

# Count duplicates
df['post_text'].duplicated().value_counts()

False    53
True     27
Name: post_text, dtype: int64

In [761]:
# Find indices of duplicate results, then drop them
duplicate_indices = df[df['post_text'].duplicated()==True].index
df_no_dups = df.drop(index=duplicate_indices)
df_no_dups = df_no_dups.reset_index(drop=True)

In [762]:
# Filter out results that don't have a price and reset index
df_with_prices = df_no_dups[df_no_dups['price']!='None']
df_with_prices = df_with_prices.reset_index(drop=True)

In [763]:
# These are the entries with 3 or more prices listed, let's investigate why
df_with_prices[df_with_prices['prices_need_cleaning']==True]['price']

0                              [30, 35, 45]
2              [40, 30, 40, 30, 40, 40, 40]
7                            [50, 100, 135]
9                              [25, 50, 25]
10    [40, 80, 40, 10, 40, 30, 40, 80, 100]
Name: price, dtype: object

In [764]:
# Inspect links manually, one by one, to decide what to do about price information
with pd.option_context('display.max_colwidth', None):
  display(df_with_prices.iloc[1]['link'])

'https://dallas.craigslist.org/dal/lss/d/new-york-ivy-grad-math-tutor-psat-sat/7420345516.html'

# Cleaning entries manually - these are the posts that had 3 or more prices listed

In [765]:
price_col_index = df_with_prices.columns.get_loc('price')
need_clean_col_index = df_with_prices.columns.get_loc('prices_need_cleaning')

### Dropping duplicates that weren't spotted earlier

These posts come up multiple times and are duplicates, but they change the text of their posting just slightly, so Pandas is unable to detect the duplicate postings on it's own, and we have to find and drop them manually.  Next, we distill their pricing schedule to a single value and mark these entries and having been cleaned.

In [766]:
# Find duplicates and drop them
kenari_tutor_indices = df_with_prices[df_with_prices['post_text'].str.contains('kenaritutor.com')==True].index

df_with_prices.drop(labels=kenari_tutor_indices[1:], inplace=True)


park_academy_indices = df_with_prices[df_with_prices['post_text'].str.contains('(949) 490-0872', regex=False)==True].index

df_with_prices.drop(labels=park_academy_indices[1:], inplace=True)



try:
    # Because the ad says $90 in person, $60 for online, and Corona Virus pricing of
    # $40 for online weekdays, I'm using the $40 per hour rate because it seems the
    # most reasonable.  We also set prices_need_cleaning to False b/c the prices have
    # been cleaned
    df_with_prices.iloc[kenari_tutor_indices[0],
                        [price_col_index,
                         need_clean_col_index]
                       ] = 40, False
except:
    pass

try:
    # The add says $55/hr for K-12, then $65/hr for AP/Honors, as well as Pre-calc, 
    # etc., I'm going to average the two prices
    df_with_prices.iloc[park_academy_indices[0],
                        [price_col_index,
                         need_clean_col_index]
                       ] = 60, False
except:
    pass

# Reset indices again
df_with_prices = df_with_prices.reset_index(drop=True)

# Find duplicates and drop them
star_star_college_math_tutor_indices = df_with_prices[df_with_prices['post_text'].str.contains('https://www.youtube.com/channel/UCqhFZRmUqOAAPMQpo58TV7g'
                   ) == True].index

df_with_prices.drop(labels=star_star_college_math_tutor_indices[1:], inplace=True)


# The ad repeats the price of $40 over and over, so I'm replacing the price with a single instance.  We also set prices_need_cleaning to False b/c the prices have been cleaned.
try:
    df_with_prices.iloc[star_star_college_math_tutor_indices[0], [price_col_index, need_clean_col_index]] = 40, False
    
except:
    pass
# Reset indices again because we dropped more rows
df_with_prices = df_with_prices.reset_index(drop=True)

### Distilling long lists of prices down to one price

Manually inspect each 

In [767]:
# This ad mentions several prices for different subjects, but explicitly says $30 for math
la_honda_indices = df_with_prices[df_with_prices['post_text'].str.contains('909-640-3570')].index

df_with_prices.iloc[la_honda_indices,[price_col_index, need_clean_col_index]] = 30, False


# This ad mentions $45 for lower division courses, which are primarily the subjects I help with, so I'm using that price to compare myself against.
ucb_phd_student_and_ta_indices = df_with_prices[df_with_prices['post_text'].str.contains('Former UC-Berkeley economics Ph.D. student and TA')].index

df_with_prices.iloc[ucb_phd_student_and_ta_indices,[price_col_index, need_clean_col_index]] = 45, False


# Says $40 for in person, or $45 for at home, so I took the average
san_mateo_tutor_indices = df_with_prices[df_with_prices['post_text'].str.contains('I mainly tutor, in person, at the Downtown Redwood City, downtown San Mateo')].index

df_with_prices.iloc[san_mateo_tutor_indices,[price_col_index, need_clean_col_index]] = 42.5, False

In [768]:
#This guy has weird price structuring, but I basically found his hourly rate for each time interval, $100 for 80 minutes, $115 for 100 minutes, $130 for 120 minutes, then averaged those hourly rates
oakland_exp_tutor_online_indices = df_with_prices[df_with_prices['post_text'].str.contains('I received a full scholarship to University of Cincinnati and held a 3.8 GPA through my master’s program in aerospace')==True].index

oakland_tutor_avg_rate = ((100/80) + (115/100) + (130/120)) * 60 / 3

df_with_prices.iloc[oakland_exp_tutor_online_indices,[price_col_index, need_clean_col_index]] = oakland_tutor_avg_rate, False


# This guy's ad says $57 per hour, but for some reason Python has trouble parsing this price on it's own earlier, so we do it ourselves.
blake_tutoring_indices = df_with_prices[df_with_prices['post_text'].str.contains('BlakeTutoring.com', case=False)==True].index

df_with_prices.iloc[blake_tutoring_indices, price_col_index] = 57

# Charges $50 per hour for sessions under 3 hours
spss_tutor_indices = df_with_prices[df_with_prices['post_text'].str.contains('Worked for 2 companies named', case=False)==True].index

df_with_prices.iloc[spss_tutor_indices, [price_col_index, need_clean_col_index]] = 50, False

In [769]:
sharp_mind_indices = df_with_prices[df_with_prices['post_text'].str.contains('(650) 398-9490', regex=False)==True].index

# Says $25/hr for high school, $30/hr for college, just went with $30/hr
df_with_prices.iloc[sharp_mind_indices, [price_col_index, need_clean_col_index]] = 30, False

trevor_skelly_indices = df_with_prices[df_with_prices['post_text'].str.contains('trevorskelly')==True].index

# Says $50/hr
df_with_prices.iloc[trevor_skelly_indices, [price_col_index, need_clean_col_index]] = 50, False

In [770]:
best_math_idx = df_with_prices[df_with_prices['post_text'].str.contains('bestmathtutoring.com')==True].index

# $25/hr if meeting near CSU Sac, $35/hr if they drive to you, $20/hr for online.
# I chose $30/hr to split the difference between the in person prices.
df_with_prices.iloc[best_math_idx, [price_col_index, need_clean_col_index]] = 30, False

glasses_lady_idx = df_with_prices[df_with_prices['post_text'].str.contains("offering virtual one-on-one Math tutoring via Zoom")==True].index

df_with_prices.iloc[glasses_lady_idx, [price_col_index, need_clean_col_index]] = 60, False

ucla_grad_henry_idx = df_with_prices[df_with_prices['post_text'].str.contains("916 390-7923", regex=False)==True].index

df_with_prices.iloc[ucla_grad_henry_idx, [price_col_index, need_clean_col_index]] = 35, False

In [771]:
peter_d_idx = df_with_prices[df_with_prices['post_text'].str.contains('Peter D.')==True].index

df_with_prices.iloc[peter_d_idx, [price_col_index, need_clean_col_index]] = 40, False

In [772]:
austin_sabrina_idx = df_with_prices[df_with_prices['post_text'].str.contains('My girlfriend Sabrina')==True].index

# $20/hr online, $30/hr in person, split the difference at $25
df_with_prices.iloc[austin_sabrina_idx, [price_col_index, need_clean_col_index]] = 25, False

In [773]:
alex_farrell_idx = df_with_prices[df_with_prices['post_text'].str.contains('Alexander Farrell')==True].index
df_with_prices.iloc[alex_farrell_idx, [price_col_index, need_clean_col_index]] = 25, False

### Checking results

In [774]:
post_with_prices_count = len(df_with_prices)
num_posts = len(df_no_dups)

percent_with_prices = post_with_prices_count/num_posts * 100

print(F"There were {post_with_prices_count} posts that had prices included.")
print(F"Only {percent_with_prices:.2f}% of the posts that we scraped had pricing information included.")

There were 12 posts that had prices included.
Only 22.64% of the posts that we scraped had pricing information included.


In [775]:
# Check to see if there are any prices with more than one value listed that would still need cleaning
df_with_prices[df_with_prices['prices_need_cleaning']==True]['price']

Series([], Name: price, dtype: object)

In [776]:
# Check to see if there are any prices with that are NaN because Python wasn't able to parse them properly according to our rules from earlier
df_with_prices[df_with_prices['price'].isnull()==True]

,date_posted,link,price,city,subregion,region,post_text,prices_need_cleaning,posts_scraped_on


In [777]:
# If any prices still need cleaning, inspect their links to decide what to do about price information
with pd.option_context('display.max_colwidth', None):
  display(df_with_prices.iloc[9]['link'])

'https://dallas.craigslist.org/dal/lss/d/arlington-tutoring-lessons-in/7423210219.html'

In [778]:
df_with_prices['region']

0     dallas
1     dallas
2     dallas
3     dallas
4     dallas
5     dallas
6     dallas
7     dallas
8     dallas
9     dallas
10    dallas
11    dallas
Name: region, dtype: object

# Saving results

In [779]:
# Store cleaned data as CSV file in preparation for importing to SQL database
df_for_sql = df_with_prices.drop(labels=['prices_need_cleaning','link'], axis=1)
df_for_sql.to_csv("./{}_{}_with_prices.csv".format(date_of_html_request, region_list[0]), index=False, sep=';')

# Store original data, before we did any manual cleaning, in case it's needed
df.to_csv("./{}_{}_all_posts.csv".format(date_of_html_request, region_list[0]), index=False)

# Importing into PostgreSQL database

In [780]:
# Establish connection to PSQL database
conn = psycopg2.connect("host=localhost dbname=rancher user=rancher")

# Instantiate a cursor object
cur = conn.cursor()

# Use cursor object to create a database for storing the information we scraped and cleaned
cur.execute("""    
    CREATE TABLE IF NOT EXISTS cl_tutoring2(
    id SERIAL primary key,
    date_scraped date,
    price decimal,
    city text,
    subregion text,
    region text,
    post_text text,
    date_posted timestamp
);
""")

# Commit changes to database
conn.commit()

In [781]:
# Instantiate a new cursor object
cur = conn.cursor()

# Copy data from our CSV file into database.  
### Note, we use the ; separator because we replaced all instances of semicolons in a post to commas, ensuring that psycopg2 won't misinterpret a semicolon in the body of a post as a separator, splitting a row in the CSV file into too many columns as a result.
### Also, we must specify null="" because Python represents null values as an empty string when writing to a CSV file and psycopg2 needs to know how null values are represented in the CSV file in order to properly insert null values into the database
with open(str(date_of_html_request) + '_' + region_list[0] + '_with_prices.csv', 'r') as file:
    next(file) # Skip the header row
    cur.copy_from(file, 'cl_tutoring2', sep=';', null="", columns=('date_posted', 'price', 'city', 'subregion', 'region', 'post_text', 'date_scraped'))
    
# Commit changes to database
conn.commit()

In [4]:
conn = psycopg2.connect("host=localhost dbname=rancher")

cur = conn.cursor()

cur.execute("""SELECT * FROM cl_tutoring2""")

results = cur.fetchall()

NameError: name 'psycopg2' is not defined

In [3]:
df = pd.read_sql_query("""SELECT * FROM cl_tutoring2""", con=conn)

NameError: name 'pd' is not defined

In [72]:
df['subregion'].value_counts()

San Francisco    11
South Bay         7
East Bay          5
Peninsula         5
North Bay         1
Santa Cruz        1
Name: subregion, dtype: int64

In [1]:
df['price'].mean()

NameError: name 'df' is not defined

In [74]:
df['price'].median()

45.0

In [726]:
results

[(1,
  datetime.date(2021, 12, 22),
  Decimal('60'),
  'Fremont / Union City / Newark',
  'East Bay',
  'sf_bay_area',
  '$60/hr.I have a broad background in the sciences, as well as skills useful for the college application process.Undergraduate: Rice University, double major Biochemistry and Physics, summa cum laude.Ph.D. Yale University, Molecular Biophysics and Biochemistry.  My Ph.D. advisor, Thomas A. Steitz, received the Nobel Prize in Chemistry, 2009.I’ve had great success helping students edit their college application essays.  I’ve taught a small writing class.  I’ve also had experience interviewing student applicants, as an alumni interviewer for Rice.As a tutor, I believe strongly in teaching students how to think critically. I prefer in-person tutoring.  I received my second Moderna shot on 4/30/2021 and the Moderna booster at the end of October.    ',
  datetime.datetime(2021, 12, 22, 15, 47, 32)),
 (2,
  datetime.date(2021, 12, 22),
  Decimal('20'),
  'None',
  'South Ba

# Scratch work

In [114]:
# sf_response = get('https://post.craigslist.org/k/avwmCB5A7BGhNo8Z2XZiFg/nQia5?s=hood')

# sf_soup = BeautifulSoup(sf_response.text, 'html.parser')

# sf_options = sf_soup.find_all(class_='option-label')

# sf_towns = []
# for town in sf_options[1:]:
#     town_str = town.get_text()
#     sf_towns.append(town_str)

In [118]:
# southbay_response = get('https://post.craigslist.org/k/QocjZx5A7BGoSgyNMfHWMg/enONz?s=hood')

# southbay_soup = BeautifulSoup(southbay_response.text, 'html.parser')

# southbay_options = southbay_soup.find_all(class_='option-label')

# southbay_towns = []
# for town in southbay_options[1:]:
#     town_str = town.get_text()
#     southbay_towns.append(town_str)

In [123]:
# eastbay_response = get('https://post.craigslist.org/k/chJu8h5A7BG1vbkl7zGu2A/zoAyD?s=hood')

# eastbay_soup = BeautifulSoup(eastbay_response.text, 'html.parser')

# eastbay_options = eastbay_soup.find_all(class_='option-label')

# eastbay_towns = []
# for town in eastbay_options[1:]:
#     town_str = town.get_text()
#     eastbay_towns.append(town_str)

In [120]:
# peninsula_response = get('https://post.craigslist.org/k/MGmMsB5A7BGZmpPWoASCRg/ZNqLC?s=hood')

# peninsula_soup = BeautifulSoup(peninsula_response.text, 'html.parser')

# peninsula_options = peninsula_soup.find_all(class_='option-label')

# peninsula_towns = []
# for town in peninsula_options[1:]:
#     town_str = town.get_text()
#     peninsula_towns.append(town_str)

In [133]:
# northbay_response = get('https://post.craigslist.org/k/EDIENx1A7BGuYAts9K5Q4g/LPMna?s=hood')

# northbay_soup = BeautifulSoup(northbay_response.text, 'html.parser')

# northbay_options = northbay_soup.find_all(class_='option-label')

# northbay_towns = []
# for town in northbay_options[1:]:
#     town_str = town.get_text()
#     northbay_towns.append(town_str)

In [122]:
# santacruzco_response = get('https://post.craigslist.org/k/DF_j3B5A7BGwdNr-8yOE1Q/xmAGM?s=hood')

# santacruzco_soup = BeautifulSoup(santacruzco_response.text, 'html.parser')

# santacruzco_options = santacruzco_soup.find_all(class_='option-label')

# santacruzco_towns = []
# for town in santacruzco_options[1:]:
#     town_str = town.get_text()
#     santacruzco_towns.append(town_str)